<a href="https://colab.research.google.com/github/Madrinmarison/Deep_Learning/blob/main/Transfer_Learning_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with Tensorflow Part 1: Features Extraction

Transfer learning - leveraging an existing working model's architecture and learned patterns  for our own problem.

Two main benefits:
1. Can leverage an existing neural network architecture proven to work on problem similar to our own.
2. Can laverage a working neural network architecture which has already learned patterns on similar data to our own, then we can adpt those patterns to our own data.

In [ ]:
# Confirm you're connected to GPU
!nvidia-smi

In [ ]:
## Get data (10% of 10 food classes from Food101)
import zipfile
# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

### Becoming one with the data

In [ ]:
# How many images in each folder
import os

# Walk through 10% data directory and list number of files.
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} files in '{dirpath}'.")

## Create data loaders (preparing the data)
Use the `ImageDataGenetator` class to load in our images in batches.

- Import the class from the `tensorflow.keras.preprocessing.image` module
- Instatiate an instance of the class and pass the class the corect attributes you would like to be incorporated while loading the image
- call a `flow_from_directoty` method and pass an appropriate parameters

In [ ]:
# Set up data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224,224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale = 1 / 255.)
test_datagen = ImageDataGenerator(rescale = 1 / 255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                  target_size=IMAGE_SIZE,
                                                  batch_size=BATCH_SIZE,
                                                  class_mode = "categorical")

print("Test images:")
test_data_10_percent = test_datagen.flow_from_directory(test_dir,
                                                        target_size=IMAGE_SIZE,
                                                        batch_size=BATCH_SIZE,
                                                        class_mode = "categorical")



## St up callbacks (things to run whislt our model train)

Callbacks are extra functionality you can add to your models to be performed during or after training. Some of te most popular callbacks are:
* Tracking an experiment with *tensorboard callback*.
* Model checkpoint with the ModelCheckpointcallback
* Stopping the model from training (before it trains for too long and overfits) with EarlyStopping callback

In [ ]:
# Create TensorBoard callback (functionized because we want to create a new one for each model)
import datetime

def creat_tensorbord_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
    print(f"Saving TensorBoard log file to: {log_dir}")
    return tensorboard_callback

You can customize the directory where your TensorBoard logs (model traing metrics) get saved to wherever you like.

The `log_dir` parameter we have created above is only optional.

## Creating models using TensorFlow Hub

In the past we've used Tensorflow to create our models layer by layer from the scratch.

Now we are going to do a similar process the majority of our model's layers are going to come from TensorFlow Hub.

We can access a pre-trained model on: https://tfhub.dev/

Browsing the TensorFlow Hub page and sorting for image classification, we found the feature vector model link provided [here](https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1).

In [ ]:
# Compare the following two models
resmet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

efficient_net = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
# Import depedencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# Create function to create a model from url
def create_model(model_url, num_classes = 10):
    """
    Takes TensorFlow Hub URL and creates keras Sequential model with it.

    Args:
        model_url (str): A TensorFlow Hub feature extraction URL.
        num_classes (int): Number of output neurons in the output layer,
            should be equal to number of target classes, defult = 10
    
    Returns:
        An uncompiled keras Sequential model with model URL as feature extractor
        layer and Dense output layer with num_classes output neuron
    """
    # Download the pretrained model and save it as a keras layer
    feature_extract_layer = hub.KerasLayer(model_url,
                                           trainable = False,
                                           name = "feature_extraction_layer",
                                           input_shape = IMAGE_SIZE + (3,)) # Freeze an already learned patterns
    # Creatae our own model
    model = tf.keras.Sequential([
        feature_extract_layer,
        layers.Dense(num_classes, activation = "softmax", name = "output_layer")
    ])
    return model
    

# Creating and testing Resnet TensorFlow Hub Feature Extraction model

In [ ]:
# Create the resenet model
resnet_model = create_model(resmet_url,
                             num_classes = train_data_10_percent.num_classes)

In [ ]:
resnet_model.summary()

In [ ]:
# compile our resenet model
resnet_model.compile(loss = "categorical_crossentropy",
                     optimizer = tf.keras.optimizers.Adam(),
                     metrics = ["accuracy"])

In [ ]:
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs = 5,
                                  steps_per_epoch = len(train_data_10_percent),
                                  validation_data = test_data_10_percent,
                                  validation_steps = len(test_data_10_percent),
                                  callbacks = [creat_tensorbord_callback(dir_name = "tensorflow_hub",
                                                                         experiment_name = "resnet_v2_50")])
                                  

This model, transfer learning feature extractor, outperformed all the previuos model we build from the scracth and in a quicker training time with only 10% of the training example.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Create a loss curve plotting function
# Tidbit: you could put a function like this into a script called "helper.py" and import it whenever needed... upload to git
def plot_loss_curves(history):
    """
    Returns seperate loss curves for training and validation metrics
    Args:
        history: TensorFlow History object.

    Returns:
        Plots of training/validation loss and accuracy metrics.
    """
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    
    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    epochs = range(len(history.history["loss"]))

    # plot loss
    plt.plot(epochs, loss, label = "training_loss")
    plt.plot(epochs, val_loss, label = "val_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend();

    # plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label = "training_accuracy")
    plt.plot(epochs, val_accuracy, label = "val_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend();

In [ ]:
plot_loss_curves(resnet_history)

# Creating and testing EfficientNetB0 TensorFlow Hub Feature Extraction model

In [ ]:
# Create EfficientNetB0 Feature extracor model
efficient_model = create_model(model_url = efficient_net,
                               num_classes = train_data_10_percent.num_classes)

# Compile the model
efficient_model.compile(loss = "categorical_crossentropy",
                        optimizer = tf.keras.optimizers.Adam(),
                        metrics = ["accuracy"])

# Create the model
efficient_history = efficient_model.fit(train_data_10_percent,
                                        epochs = 5,
                                        steps_per_epoch = len(train_data_10_percent),
                                        validation_data = test_data_10_percent,
                                        validation_steps = len(test_data_10_percent),
                                        callbacks = [creat_tensorbord_callback(dir_name = "tensorflow_hub",
                                                                         experiment_name = "efficientnetb0")])

In [ ]:
# plot loss and accuracy curves
plot_loss_curves(efficient_history)

In [ ]:
# Create EfficientNetB0 Feature extracot model
efficient_model_1 = create_model(model_url = efficient_net,
                               num_classes = train_data_10_percent.num_classes)

# Compile the model
efficient_model_1.compile(loss = "categorical_crossentropy",
                        optimizer = tf.keras.optimizers.Adam(),
                        metrics = ["accuracy"])

# Create the model
efficient_history_1 = efficient_model_1.fit(train_data_10_percent,
                                        epochs = 28,
                                        steps_per_epoch = len(train_data_10_percent),
                                        validation_data = test_data_10_percent,
                                        validation_steps = len(test_data_10_percent))
                                        #callbacks = [creat_tensorbord_callback(dir_name = "tensorflow_hub",
                                                                       #  experiment_name = "efficientnetb0")])

# Different types of transfer learning

* **"As is" transfer learning:** - using an existing model with no changes what so ever (e.g. using ImageNet model on 1000 image classes, on of your own problem)
* **Feature extraction" transefer learning:** -  Use pre-learned patterns of an existing model (e.g using EfficientB0 trained on ImageNet) and adjust the output layer for your probelem (e.g. 1000 classes --> 10 classes of food)
* **"Fine-tunning" transfer learning**

# Comparing model's results using TensorBoard
> 🔑 **Note:** When you upload things to TensorBoard.dev, your experiments are public. So if you're running private experiment (things you don't want others to see) do not upload them to TensorBoard.dev.

In [ ]:
# Upload TensorBoadr dev records
!tensorboard dev upload --logdir ./tensorflow_hub/ \
--name "EfficientB0 vs ResNet50V2" \
--description "Compairing two different TF Hub feature extraction model architecture using 10% of the training data" \
--one_shot

Our TensorFlow experiment are uploaded pulically [here](https://tensorboard.dev/experiment/WpVXZHr7TMqWqpVJtz9ZAg/)

In [ ]:
# check out what TensorBoard experiments you have
!tensorboard dev list

In [ ]:
# Delete an experiment from TensorBoard
# !tensorboard dev delete --experiment_id WpVXZHr7TMqWqpVJtz9ZAg

In [ ]:
# Confirn the deletion by rechecking what experiments you have left
!tensorboard dev list